In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [6]:
glove_wiki_vectors.most_similar("king")

[('prince', 0.7682329416275024),
 ('queen', 0.7507689595222473),
 ('son', 0.7020888328552246),
 ('brother', 0.6985775828361511),
 ('monarch', 0.6977890729904175),
 ('throne', 0.691999077796936),
 ('kingdom', 0.6811409592628479),
 ('father', 0.680202841758728),
 ('emperor', 0.67128586769104),
 ('ii', 0.6676074266433716)]

In [7]:
glove_wiki_vectors.most_similar("cluster")

[('clusters', 0.7728561162948608),
 ('bomblets', 0.6050967574119568),
 ('globular', 0.5860958695411682),
 ('bombs', 0.5577062964439392),
 ('small', 0.5404573678970337),
 ('shells', 0.5340219736099243),
 ('munitions', 0.5319870710372925),
 ('unexploded', 0.531724214553833),
 ('large', 0.529614269733429),
 ('mass', 0.5238727331161499)]

In [8]:
glove_wiki_vectors.most_similar("ubc")

[('unm', 0.588445246219635),
 ('ucl', 0.5795673131942749),
 ('dalhousie', 0.5718241930007935),
 ('unsw', 0.562849760055542),
 ('wvu', 0.5414904356002808),
 ('optometry', 0.5404933094978333),
 ('bsu', 0.538948118686676),
 ('uvm', 0.5379002690315247),
 ('vcu', 0.5366261601448059),
 ('fiu', 0.536444902420044)]

In [9]:
glove_wiki_vectors.most_similar("currency")

[('currencies', 0.8316828608512878),
 ('dollar', 0.7609074711799622),
 ('monetary', 0.732880175113678),
 ('markets', 0.7306826710700989),
 ('euro', 0.7265002727508545),
 ('exchange', 0.6967350840568542),
 ('ruble', 0.6959021091461182),
 ('market', 0.6943885684013367),
 ('devaluation', 0.6870244145393372),
 ('peso', 0.6787100434303284)]

In [10]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [12]:
similarity_matrix = []

for w1, w2 in word_pairs:
    similarity_matrix.append(glove_wiki_vectors.similarity(w1, w2))
    
similarity_matrix
#outputs cosine similarities for each word pair 

[0.70002717, 0.54627603, 0.64324874, 0.7552733, 0.87980753, 0.076719455]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.  

In [13]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [14]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [15]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [16]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

In [17]:
analogy("man", "smart", "woman").head(7)

man : smart :: woman : ?


,Analogy word,Score
0,intelligent,0.654885
1,sexy,0.597897
2,sophisticated,0.574362
3,mom,0.557256
4,cute,0.549813
5,kids,0.540299
6,pretty,0.530862


In [18]:
glove_wiki_vectors.similarity("man", "rich")

0.46977922

In [19]:
glove_wiki_vectors.similarity("female", "rich")

0.2790344

Biases are still definitely present and brought to light in this dataset, most drastically with the example of comparing words "female" and "rich", where the score was quite low - highlighting the gender wage gap between females and males in the workforce. Additionally, in our analogy with "man", "smart", and "woman" physical characteristics were used such as 'sexy', 'pretty', and 'cute' rather than characteristics that are actually associated with intelligence.

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

1. Using pre-trained embeddings is a really powerful tool that we can leverage to make predictions on difficult data (free-form text data in particular), but we should be careful about using the embeddings as well since they may inadvertently have biases and stereotypes embedded in them, sometimes even without are knowledge. So it's important to always keep in mind how the results we generate from our machine learning models are being generated and identify whether or not they could be caused by biases in our data or in our pre-trained embeddings in this case.
2. Such a model could be harmful in the real world because it teaches data models to reduce gender minorities to superficial characteristics - not only can this harm both society's perception on gender minorities and their self-perception, but it reinforces the idea that gender minorities deserve to be paid less as they are taken less seriously than white male leaders even though they are contributing the same work and may have the same qualifications, and it hence affects their livelihood.  With Machine Learning and NLP we have the opportunity to systemically break gender stereotypes by producing unbiased results, rather than reinforcing society's gender laws from regular day life. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Classification with pre-trained embeddings 
rubric={points:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`) and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. 
2. Train logistic regression with average embedding representation extracted using spaCy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [20]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [21]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [22]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [23]:
import spacy

nlp = spacy.load("en_core_web_md")

<div class="alert alert-warning">

Solution_1.5
    
</div>

In [24]:
from sklearn.metrics import classification_report

In [25]:
#1. Train logistic regression with bag-of-words features (CountVectorizer) and show classification report on the test set.

pipe = make_pipeline(
    CountVectorizer(), LogisticRegression(max_iter=1000)
)
pipe.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = pipe.named_steps["countvectorizer"].transform(X_train)
pipe.fit(X_train, y_train)

print(
    classification_report(
        y_test, pipe.predict(X_test), target_names=["affection", "exercise", "bonding", "nature", "leisure", "achievement", "enjoy_the_moment"]
    )
)

                  precision    recall  f1-score   support

       affection       0.80      0.85      0.82      1302
        exercise       0.91      0.92      0.92      1423
         bonding       0.89      0.86      0.87       492
          nature       0.59      0.54      0.56       469
         leisure       0.91      0.55      0.69        74
     achievement       0.71      0.72      0.72       407
enjoy_the_moment       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.79      0.70      0.74      4238
    weighted avg       0.82      0.82      0.82      4238



In [26]:
#2. Train logistic regression with average embedding representation extracted using spaCy and show classification report on the test set.

X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

lgr = LogisticRegression(max_iter=5000)
lgr.fit(X_train_embeddings, y_train)

print(
    classification_report(
        y_test, lgr.predict(X_test_embeddings), target_names=["affection", "exercise", "bonding", "nature", "leisure", "achievement", "enjoy_the_moment"]
    )
)

                  precision    recall  f1-score   support

       affection       0.81      0.83      0.82      1302
        exercise       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
          nature       0.57      0.51      0.54       469
         leisure       0.68      0.76      0.72        74
     achievement       0.72      0.65      0.68       407
enjoy_the_moment       0.69      0.72      0.70        71

        accuracy                           0.79      4238
       macro avg       0.74      0.74      0.74      4238
    weighted avg       0.79      0.79      0.79      4238



In [27]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. The difference is that using CountVectorizer (bag-of-words) simply counts the instances of a words in the provided text, whereas average-embedding we find complex word vectors from the pretrained embeddings of spaCy and compute the average of these to represent the text data. 
2. Bag of words is a more interpretable representation compared to average embedding because it is kept in an english form and is therefore more readable/interpretable to humans. This is more interpretable than average embedding, where text data is represented by vectors.
3. Yes, since embeddings are trained on the spaCy model's corpus. In this case the CountVectorizer strategy in general appears to perform marginally better, but we notice that average-embedding is less overfitting. Thus we can assume that there is benefit in that transfer learning would generalize better to unseen data. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [28]:
from sklearn.datasets import fetch_20newsgroups

In [29]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [30]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

`code adapted from cpsc 330 lecture 18`

In [31]:
import spacy

nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

In [32]:
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE", "SYM"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            
            token.is_stop == False  # Check if it's not a stopword
            and token.like_url == False  # Check if it looks like a url
            and token.like_email == False  # Check if it looks like an email
            
            and len(token) > min_token_len  # Check if it meets minimum threshold
            and token.pos_ not in irrelevant_pos # Check if it is part of the irrelevent part-of-speach tag set 
            
        ):  
            
            # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
            
    return " ".join(clean_text)

In [33]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]
df["text_pp"]

0       know read u.s.c. 922 sence wonder help u.s.c. ...
1       bad question ref list algorithm think bit hard...
2       actuallay hand support idea have newsgroup asp...
3       following problem bug appreciate help create w...
4       late upi foreign ministry spokesman ferhat ata...
                              ...                        
4558    look software call shadow know simple raytrace...
4559    archive faq part3 modify 1993/04/04 subject pa...
4560    nice answer question difference feds mandate l...
4561    get gateway 4dx-33v try configure x11r5 need c...
4562    armenians nagarno karabagh defend right homela...
Name: text_pp, Length: 4563, dtype: object

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

- Removing stopwords - by removing words that are very common in the English language , we remove low-level information from our text in order to give more focus to the important information. 
- Remove urls - urls are not relevant for our analysis of topics  
- Remove emails - emails are not relevant for our analysis of topics
- length > 2 - discarding words with length < threshold or word frequency of 2 to give us weightier tokens, otherwise we could have just used bag-of-words for simplicity
- Removing adverbs, pronouns, coordinating conjunctions, punctuation, particles, determiner adpositions, spaces, symbols - to restrict to only particular aspects of speech that give useful semantic information.
- Lemmatization - aims to only consider lemmas (base form of the vocab word) rather than inflections. 
- Lowercase - aims to simplify the text and maintain consistency.

Combining all these preprocessing steps together leads us to a cleaner version of the text data that we can now perform topic modeling on.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

`code adapted from cpsc 330 lecture 18`

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

In [35]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4

lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)

document_topics = lda.fit_transform(X)

We chose the value of `n_topic = 4` for n_components because from our selected categories we can infer four topic clusters: sports, religion, computers, and politics. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

`code adapted from cpsc 330 lecture 18`


In [36]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [37]:
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
god           people        file          game          
know          say           program       team          
jesus         know          use           year          
believe       think         image         gun           
think         come          window        play          
people        time          available     good          
say           armenian      include       player        
time          right         entry         think         
good          kill          run           win           
thing         state         server        season        




- Topic 1: Religion
- Topic 2: Politics
- Topic 3: Computers 
- Topic 4: Sports

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [38]:
document_topics[0:5]

array([[0.0049272 , 0.93237069, 0.00478888, 0.05791322],
       [0.17494391, 0.36215434, 0.36743913, 0.09546263],
       [0.28529384, 0.41272216, 0.29729762, 0.00468638],
       [0.00893355, 0.78919188, 0.19318493, 0.00868963],
       [0.00598984, 0.98209716, 0.00598773, 0.00592528]])

In [39]:
docs = df[["target_name", "text_pp"]][0:5]
doc_topics = ["politics", "computers", "politics", "politics", "politics"]
docs = docs.assign(topics=doc_topics)
docs

,target_name,text_pp,topics
0,talk.politics.guns,know read u.s.c. 922 sence wonder help u.s.c. ...,politics
1,comp.graphics,bad question ref list algorithm think bit hard...,computers
2,comp.graphics,actuallay hand support idea have newsgroup asp...,politics
3,comp.windows.x,following problem bug appreciate help create w...,politics
4,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...,politics


- Document 1: the model assigns it to topic 2 (politics) with very high confidence. 
- Document 2: the model assigns it to topic 3 (computers), but there's very low confidence (prefers the politics topic almost as much).
- Document 3: the model assigns it to topic 2 (politics), and confidence is moderate (beats topic 1 and topic 3 (computers) which are roughly equal).
- Document 4: the model assigns it to topic 2 (politics) with high confidence. 
- Document 5: the model assigns it to topic 2 (politics) with very high confidence.

So in summary we can see that the document topic assignment model seems to be capturing some of the text posts into the appropriate clusters, although the model we've built seems to posts are sometimes about politics even when they are not (although we can argue that maybe this is a reflective of the dataset (maybe people post about politics more often than they do computers, and our model has learned this bias).

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

1.
- Decision Tree - one vs rest as there will be one binary classifier for each class, whichever classifier comes back with the highest value, the example will be assigned that class.
- KNN - one vs one approach for multi-class classification. 
- Random Forest -  one vs rest as there will be one binary classifier for each class, like decision trees.
- Logistic Regression - uses the one-vs-rest (OvR) scheme if the ‘multi_class’ option is set to ‘ovr’, and uses the cross-entropy loss if the ‘multi_class’ option is set to ‘multinomial’
- SVM RBF - implements the “one-versus-one” approach for multi-class classification. In total, n_classes * (n_classes - 1) / 2 classifiers are constructed and each one trains data from two classes
2. Transfer learning is the process of using a pre-trained model for solving your task either directly or by fine tuning it. Instead of developing deep networks (CNN or NLP models) from scratch, you can use transfer learning in one of three ways: 
    a. use a pre-trained model out of the box
    b. use a pre-trained model for extracting features that you can then use to train your model
    c. use a pre-trained models with the default weights and fine-tune them for your task. 
3. This property is useful for transfer learning because we can easily manipulate the sequential transformations to better suit the requirements of our task (fine-tuning). 

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)